In [44]:
import pandas as pd

In [45]:
dir_path='J:\\NanTongAPTIV\\newTable\\'
material_fileName='material.xlsx'
mod_resource_fileName='mod_resource.xlsx'
shift_mod_fileName='可变模具20230412.xlsx'
material_path=dir_path+material_fileName
mod_resource_path=dir_path+mod_resource_fileName
shift_mod_path=dir_path+shift_mod_fileName
material=pd.read_excel(material_path)


In [46]:
#拿模具的excel数据
mod_resource=pd.read_excel(mod_resource_path)

In [47]:
#拿material表的
material=pd.read_excel(material_path)

In [58]:
#拿可变模具的excel
shift_mod_group=pd.read_excel(shift_mod_path,sheet_name='sheet1')
shift_mod_group_bak=pd.read_excel(shift_mod_path,sheet_name='sheet1')

In [59]:
#最终要出现的表的样式
shift_mod_efficiency=pd.DataFrame(columns=['resource_code','source_material_code','target_material_code','shift_time'])
err_mod_message=pd.DataFrame(columns=['resource_code','material_code','shift_time'])
err_material_message=pd.DataFrame(columns=['resource_code','material_code','shift_time'])
err_mod_has_one_material_message=pd.DataFrame(columns=['resource_id','material_id','shift_time'])
# err_mod_message

In [60]:
#数据格式准备
shift_mod_group.rename(columns={'资源编号':'resource_code','物料':'material_code','轮换时间(分钟)':'shift_time'},inplace=True)
shift_mod_group_bak.rename(columns={'资源编号':'resource_code','物料':'material_code','轮换时间(分钟)':'shift_time'},inplace=True)
material_efficiency=material[['id','code']]
mod_resource_efficiency=mod_resource[['id','code']]

In [ ]:
# 第一步数据查错看看material_code的数据有没有重复的

In [61]:
for i in range(1,len(shift_mod_group)):
    temp_mod_resource_code_to_id=pd.DataFrame(columns=['id','code'])
    # 拿到resource_id
    temp_mod_resource_code_to_id=mod_resource_efficiency[shift_mod_group['resource_code'][i] == mod_resource_efficiency['code']]
    if (temp_mod_resource_code_to_id.empty):
        err_mod_message=pd.concat([err_mod_message,shift_mod_group.iloc[i,:].to_frame().T])
        shift_mod_group['resource_code'][i]=None
        continue
    shift_mod_group['resource_code'][i]=temp_mod_resource_code_to_id['id'].values[0]
shift_mod_group=shift_mod_group.dropna(subset=['resource_code'])
shift_mod_group.rename(columns={'resource_code':'resource_id'},inplace=True)

In [62]:
shift_mod_group.rename(columns={'resource_id':'resource_code'},inplace=True)
#先将sheet2的material_code全转成material_id
# 还需增加将错误数据导出到另一张表的操作
try:
    for i in shift_mod_group.index:
        if i==0:
            continue
        # 这里后面要加防御，避免出现这里的code不在material里面的情况
        temp_material_code_to_id=pd.DataFrame(columns=['id','code'])
        temp_material_code_to_id=material_efficiency[shift_mod_group['material_code'][i]==material_efficiency['code']]
        if(temp_material_code_to_id.empty):
            err_material_message=pd.concat([err_material_message,shift_mod_group_bak.iloc[i,:].to_frame().T])
            shift_mod_group['material_code'][i]=None
            continue
        shift_mod_group['material_code'][i]=temp_material_code_to_id['id'].values[0]
except Exception as e:
    print(i)
    print(e)
shift_mod_group=shift_mod_group.dropna(subset=['material_code'])
shift_mod_group.rename(columns={'material_code':'material_id','resource_code':'resource_id'},inplace=True)

In [53]:
err_mod_message.to_excel(dir_path+'模具编码出错.xlsx')
err_material_message.to_excel(dir_path+'物料编码出错.xlsx')

In [63]:
#这一步要不要再议
temp_shift_mod_memo_idnex=shift_mod_group
shift_mod_group=shift_mod_group.reset_index(drop=True)
# shift_mod_group

In [66]:
i=1
while (i<len(shift_mod_group)):
# for i in range(1,len(shift_mod_group)):
        #根据resource_id拿到这一组别有哪些数据,存在temp_shift_mod_group
    temp_shift_mod_group=shift_mod_group[shift_mod_group['resource_id'][i]==shift_mod_group['resource_id']]
    if len(temp_shift_mod_group)==1 or shift_mod_group['resource_id'][i]==None:
        err_mod_has_one_material_message=pd.concat([err_mod_has_one_material_message,shift_mod_group.iloc[i,:].to_frame().T])
        shift_mod_group['resource_id'][i]=None
        i+=1
        continue
    #上面拿到的只有一条的时候直接删除，不要犹豫，犹豫就会败北
    temp_shift_mod_group=temp_shift_mod_group.reset_index(drop=True)
    #这是最终所需要的表
    temp_shift_mod=pd.DataFrame(columns=['resource_code','source_material_code','target_material_code','shift_time'],index=range(len(temp_shift_mod_group)*(len(temp_shift_mod_group)-1)))
    temp_index=0
    for j in range(len(temp_shift_mod_group)):
        #当下标比这个A(n,2)=n*(n-1)大的时候
        if(temp_index>(len(temp_shift_mod_group)*(len(temp_shift_mod_group)-1))):
            break
        for s in range(len(temp_shift_mod_group)):
            # temp_shift_mod['resource_code'][temp_index]=shift_mod_group['resource_id'][i]
            temp_shift_mod['resource_code'][temp_index]=temp_shift_mod_group['resource_id'][0]
            # 从sheet1拿到shift_time，对于同一组的它的shift_time应该是不变的
            temp_shift_mod['shift_time'][temp_index]=temp_shift_mod_group['shift_time'][0]
            # 做len(material_group)次循环，每次拿到一个作为source_material_id，剩下的作为target_material_id
            temp_shift_mod['source_material_code'][temp_index]=temp_shift_mod_group['material_id'][j]
            if(s==j):
                continue
            else:
                temp_shift_mod['target_material_code'][temp_index]=temp_shift_mod_group['material_id'][s]
                temp_index+=1
    #在temp_shift_mod里面已经有A(N,2)条数据了，这时候就和最终的表进行拼接
    shift_mod_efficiency=pd.concat([shift_mod_efficiency,temp_shift_mod],ignore_index=True)
    i+=len(temp_shift_mod_group)
err_mod_has_one_material_message.rename(columns={'material_id':'material_code','resource_id':'resource_code'},inplace=True)
# err_mod_has_one_material_message.to_excel(dir_path+'模具只对应一条物料.xlsx',sheet_name='sheet1', index=False)
#尽善尽美之让它和原格式保持一致，即第一行是中文名 第二行是英文名
temp=pd.DataFrame(columns=['resource_code','source_material_code','target_material_code','shift_time'],data=[['resource_code','source_material_code','target_material_code','shift_time']])
shift_mod_efficiency=pd.concat([temp,shift_mod_efficiency],ignore_index=True)
shift_mod_efficiency.rename(columns={'resource_code':'资源编号','source_material_code':'源物料编号','target_material_code':'目标物料编号','shift_time':'轮换时间(分钟)'},inplace=True)
shift_mod_efficiency.to_excel(dir_path+'可变模具转换后.xlsx',sheet_name='sheet1', index=False)

In [75]:
#对于第三块的错误，是因为之前的操作导致一个模具只剩下一个这里我需要将id从新变回code然后做一次对应
temp_err=pd.DataFrame(columns=['resource_code','material_code','shift_time'])
for i in err_mod_has_one_material_message.index:
    temp_err_index=temp_shift_mod_memo_idnex[err_mod_has_one_material_message['material_code'][i]==temp_shift_mod_memo_idnex['material_id']].index
    temp_err=pd.concat([temp_err,shift_mod_group_bak.iloc[temp_err_index,:]])
err_mod_has_one_material_message.to_excel(dir_path+'模具只对应一条物料(转换为id版).xlsx',sheet_name='sheet1', index=False)
temp_err.to_excel(dir_path+'模具只对应一条物料(原数据版).xlsx',sheet_name='sheet1')
